See comments on generateTestFeatures (procedure is the same)

In [2]:
import importlib, os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.display import display
from icecream import ic

sys.path.append('../')
sys.path.append('/Volumes/Lab/Users/scooler/classification/')
sys.path.append("/Volumes/Lab/Users/mads/artificial-retina-software-pipeline/artificial-retina-software-pipeline/utilities/")
sys.path.append("/Volumes/Lab/Users/mads/cell_class/moosa_share/")

import cell_display_lib as cdl
import features as feat
import features_visual as feat_v
import features_electrical as feat_e
import deduplication
import features_DLelec as feat_dl

import features
import file_handling
from scipy.signal import spectrogram 
import scipy.signal as signal
import plotly.express as px
from sklearn.decomposition import PCA
import visionloader as vl
from conduction_velocity_code import get_axonal_conduction_velocity, upsample_ei, filter_ei_for_electrode_types
from scipy import stats
import eilib as el
import math
from skimage import measure
from cell_display_lib import show
import cv2
import re
pd.set_option('display.max_rows', None)

print('modules loaded! GO TIME')

# Your custom parameters
load_file = False
save_file = True
save_name = 'featExtract_test'  # customize this!
scratch_file_root = '/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL' # replace my name!

/tmp/ipykernel_68017/1875288024.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


modules loaded! GO TIME


## LOAD in Dataset Lists

In [563]:
# Gorish + Publishable Test Sets (512-array, vision)
dtlist = pd.read_csv('traintrainval_sets.csv')
dtlist.columns = ['piece','path']
dtlist['run'] = '000'
dtlist['sorter'] = 'vision'
for ind in range(len(dtlist)):
    dtlist.run[ind] = re.search(r'\d+', dtlist.path[ind][::-1]).group()[::-1]
    if 'kilosort' in dtlist.path[ind]:
        dtlist.sorter[ind] = 'kilosort'
    elif 'yass' in dtlist.path[ind]:
        dtlist.sorter[ind] = 'yass'
    # Clean up some errors in csv 
    if dtlist.piece[ind] == '2005-08-08-0':
        dtlist.path[ind] = '/Volumes/Analysis/2005-08-08-0/CT_EI_Analysis_Gorish/data010'
        dtlist.run[ind] = '010'
    if dtlist.piece[ind] == '2005-09-09-0':
        dtlist.path[ind] = 'Volumes/Analysis/2005-09-09-0/data005-auto-vision'
        dtlist.run[ind] = '005'
    if dtlist.piece[ind] == '2006-11-08-2':
        dtlist.path[ind] = 'Volumes/Analysis/2011-11-08-2/data013-dl'
        dtlist.run[ind] = '013'
    if dtlist.piece[ind] == '2006-11-08-2':
        dtlist.path[ind] = '2006-11-08-2/data013-dl'
        dtlist.run[ind] = '013'
    if dtlist.piece[ind] == '2008-03-25-5':
        dtlist.path[ind] = '2008-03-25-5/data000'
        dtlist.run[ind] = '000'
    if dtlist.piece[ind] == '2008-03-25-5':
        dtlist.path[ind] = '2008-03-25-5/data000'
        dtlist.run[ind] = '000'
    if dtlist.piece[ind] == '2011-08-01-2':
        dtlist.run[ind] = '000'
    if dtlist.piece[ind] == '2008-04-22-2':
        dtlist.path[ind] = '2008-04-22-2/CT_EI_Analysis_Gorish/data215'
    if dtlist.piece[ind] == '2011-08-04-4':
        dtlist.run[ind] = '000'
    if dtlist.piece[ind] == '2011-08-04-6':
        dtlist.run[ind] = '002'
    if dtlist.piece[ind] == '2011-08-04-7':
        dtlist.run[ind] = '000'
    if dtlist.piece[ind] == '2011-08-04-8':
        dtlist.run[ind] = '000'
    if dtlist.piece[ind] == '2011-10-25-0':
        dtlist.run[ind] = '008'
    if dtlist.piece[ind] == '2011-12-04-1':
        dtlist.run[ind] = '008'
    if dtlist.piece[ind] == '2011-12-13-2':
        dtlist.run[ind] = '023'

print(dtlist)

             piece  \
0     2003-08-06-3   
1     2003-09-19-0   
2     2003-09-19-3   
3     2004-03-19-0   
4     2004-07-30-0   
5     2004-07-30-2   
6     2005-04-06-3   
7     2005-04-06-5   
8     2005-04-14-0   
9     2005-04-14-1   
10    2005-04-14-2   
11    2005-04-14-3   
12    2005-04-14-4   
13    2005-04-19-0   
14    2005-04-19-1   
15    2005-04-19-2   
16    2005-04-19-3   
17    2005-04-21-0   
18    2005-04-21-2   
19    2005-04-26-1   
20    2005-04-26-2   
21    2005-05-24-0   
22    2005-05-24-1   
23    2005-05-24-2   
24    2005-05-24-3   
25    2005-05-26-0   
26    2005-05-26-1   
27    2005-05-26-2   
28    2005-05-26-3   
29    2005-05-26-4   
30    2005-05-26-5   
31    2005-05-26-6   
32    2005-05-26-7   
33    2005-05-31-0   
34    2005-05-31-1   
35    2005-05-31-2   
36    2005-05-31-3   
37    2005-06-02-0   
38    2005-06-02-2   
39    2005-06-02-3   
40    2005-06-07-0   
41    2005-06-07-1   
42    2005-06-07-2   
43    2005-06-07-3   
44    2005

In [564]:
dtlist = dtlist.drop([457]) # make paths error

## Set Up Dataset_Table

In [574]:
print(f'Setting up {len(dtlist)} pieces')
index = []
dataset_dict = cdl.new_dataset_dict()
# sorter_name = 'kilosort'
for ind, row in dtlist.iterrows():
    piece_id, run_id = dtlist.piece[ind], dtlist.run[ind]
    path = (f'/Volumes/Analysis/' + dtlist.path[ind])
    if not os.path.isdir(path):
        print(f'Missing main vision analysis data {path}')
        continue
    label_mode, label_data_path, sta_path, ei_path = cdl.make_paths(piece_id, run_id)
    ei_path = ''
    # label_data_path = ''
    dataset_dict['run_id'].append(run_id)
    dataset_dict['piece_id'].append(piece_id)
    dataset_dict['note'].append('novel')
    dataset_dict['path'].append(path)
    dataset_dict['labels'].append(label_mode)
    dataset_dict['sorter'].append(dtlist.sorter[ind])
    dataset_dict['label_data_path'].append(label_data_path)
    dataset_dict['sta_path'].append(sta_path)
    dataset_dict['ei_path'].append(ei_path)
    dataset_dict['stimulus_type'].append('whitenoise')
    dataset_dict['species'].append('macaque')
    index.append((piece_id, run_id))


dataset_table = pd.DataFrame(dataset_dict, index=pd.MultiIndex.from_tuples(index, names=['piece_id','run_id']))

# display(dataset_table)

Setting up 524 pieces
Missing 50-frame STA /Volumes/Scratch/Users/alexth/supersample-stas/2003-08-06-3/kilosort_data002/data002/data002.wu_sta
Missing long EI /Volumes/Scratch/Users/alexth/supersample-stas/2003-08-06-3/kilosort_data002/data002/data002.ei
Missing 50-frame STA /Volumes/Scratch/Users/alexth/supersample-stas/2003-09-19-0/kilosort_data000/data000/data000.wu_sta
Missing long EI /Volumes/Scratch/Users/alexth/supersample-stas/2003-09-19-0/kilosort_data000/data000/data000.ei
Missing 50-frame STA /Volumes/Scratch/Users/alexth/supersample-stas/2003-09-19-3/kilosort_data000/data000/data000.wu_sta
Missing long EI /Volumes/Scratch/Users/alexth/supersample-stas/2003-09-19-3/kilosort_data000/data000/data000.ei
Missing 50-frame STA /Volumes/Scratch/Users/alexth/supersample-stas/2004-03-19-0/kilosort_data000/data000/data000.wu_sta
Missing long EI /Volumes/Scratch/Users/alexth/supersample-stas/2004-03-19-0/kilosort_data000/data000/data000.ei
Missing 50-frame STA /Volumes/Scratch/Users/al

In [575]:
# Drop datasets that throw errors without quick fix
pd.options.display.max_rows = 4000
dataset_table = dataset_table.drop(index=('2006-11-17-0')) #Array Error
dataset_table = dataset_table.drop(index=('2006-11-08-2')) #Array Error
dataset_table = dataset_table.drop(index=('2006-11-08-1')) #Array Error 
dataset_table = dataset_table.drop(index=('2006-11-01-1')) #Array Error 
dataset_table = dataset_table.drop(index=('2006-09-22-2')) #Array Error
dataset_table = dataset_table.drop(index=('2003-09-19-0')) #Array Error
dataset_table = dataset_table.drop(index=('2004-03-19-0')) #Array Error
dataset_table = dataset_table.drop(index=('2005-04-14-1')) #Array Error
dataset_table = dataset_table.drop(index=('2005-04-14-2')) #Array Error
dataset_table = dataset_table.drop(index=('2005-04-19-0')) #Array Error
dataset_table = dataset_table.drop(index=('2005-04-19-1')) #Array Error
dataset_table = dataset_table.drop(index=('2005-04-19-2')) #Array Error
dataset_table = dataset_table.drop(index=('2005-04-21-2')) #Array Error
dataset_table = dataset_table.drop(index=('2005-05-24-0')) #Array Error
dataset_table = dataset_table.drop(index=('2005-05-26-2')) #Array Error
dataset_table = dataset_table.drop(index=('2005-05-26-4')) #Array Error
dataset_table = dataset_table.drop(index=('2005-05-26-6')) #Array Error
dataset_table = dataset_table.drop(index=('2005-05-31-2')) #Array Error
dataset_table = dataset_table.drop(index=('2005-07-07-0')) #Array Error
dataset_table = dataset_table.drop(index=('2005-07-26-1')) #Array Error
dataset_table = dataset_table.drop(index=('2005-08-03-1')) #Array Error
dataset_table = dataset_table.drop(index=('2005-09-09-3')) #Array Error
dataset_table = dataset_table.drop(index=('2005-09-09-4')) #Array Error
dataset_table = dataset_table.drop(index=('2005-09-20-2')) #Array Error
dataset_table = dataset_table.drop(index=('2006-04-24-0')) #Array Error
dataset_table = dataset_table.drop(index=('2006-04-24-1')) #Array Error
dataset_table = dataset_table.drop(index=('2006-04-24-2')) #Array Error
dataset_table = dataset_table.drop(index=('2006-04-24-4')) #Array Error
dataset_table = dataset_table.drop(index=('2006-04-26-0')) #Array Error
dataset_table = dataset_table.drop(index=('2006-04-26-1')) #Array Error
dataset_table = dataset_table.drop(index=('2006-05-04-4')) #Array Error
dataset_table = dataset_table.drop(index=('2006-05-05-0')) #Array Error
dataset_table = dataset_table.drop(index=('2006-05-05-1')) #Array Error
dataset_table = dataset_table.drop(index=('2006-05-22-2')) #Array Error
dataset_table = dataset_table.drop(index=('2006-05-22-5')) #Array Error
dataset_table = dataset_table.drop(index=('2006-06-02-1')) #Array Error
dataset_table = dataset_table.drop(index=('2006-06-06-0')) #Array Error
dataset_table = dataset_table.drop(index=('2006-06-06-3')) #Array Error
dataset_table = dataset_table.drop(index=('2006-06-14-0')) #Array Error
dataset_table = dataset_table.drop(index=('2006-06-26-0')) #Array Error
dataset_table = dataset_table.drop(index=('2006-06-28-1')) #Array Error
dataset_table = dataset_table.drop(index=('2006-07-05-1')) #Array Error
dataset_table = dataset_table.drop(index=('2006-07-07-1')) #Array Error
dataset_table = dataset_table.drop(index=('2006-07-07-2')) #Array Error
dataset_table = dataset_table.drop(index=('2006-07-10-2')) #Array Error
dataset_table = dataset_table.drop(index=('2006-07-12-1')) #ACF Error
dataset_table = dataset_table.drop(index=('2006-07-12-2')) #Array Error
dataset_table = dataset_table.drop(index=('2006-07-14-0')) #Array Error
dataset_table = dataset_table.drop(index=('2006-11-17-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2006-11-20-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2006-11-20-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2006-12-04-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2006-12-04-2')) #Array Error
dataset_table = dataset_table.drop(index=('2007-01-23-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2007-02-06-5')) #Array Error 
dataset_table = dataset_table.drop(index=('2007-03-01-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2007-03-08-2')) #Array Error
dataset_table = dataset_table.drop(index=('2007-03-27-3')) #EI Calc needed
dataset_table = dataset_table.drop(index=('2007-08-21-4')) #Array Error 
dataset_table = dataset_table.drop(index=('2007-08-24-4')) #Array Error
dataset_table = dataset_table.drop(index=('2007-08-24-5')) #EI Calc needed
dataset_table = dataset_table.drop(index=('2007-08-24-6')) #Array Error
dataset_table = dataset_table.drop(index=('2008-03-25-0')) #Array Error
dataset_table = dataset_table.drop(index=('2008-03-25-1')) #Array Error
dataset_table = dataset_table.drop(index=('2008-03-25-5')) #Array Error
dataset_table = dataset_table.drop(index=('2008-03-25-6')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-03-25-7')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-04-08-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-04-08-1')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-04-08-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-04-16-0')) #Array Error
dataset_table = dataset_table.drop(index=('2008-04-16-1')) #Array Error
dataset_table = dataset_table.drop(index=('2008-04-16-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-04-22-0')) #Array Error
dataset_table = dataset_table.drop(index=('2008-04-22-2')) #Array Error
dataset_table = dataset_table.drop(index=('2008-04-22-3')) #Array Error
dataset_table = dataset_table.drop(index=('2008-04-22-5')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-04-22-6')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-04-30-1')) #Array Error
dataset_table = dataset_table.drop(index=('2008-04-30-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-04-30-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-05-13-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-05-13-1')) #EI calc needed
dataset_table = dataset_table.drop(index=('2008-05-13-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-05-13-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-06-10-0')) #Array Error
dataset_table = dataset_table.drop(index=('2008-06-10-1')) #Array Error
dataset_table = dataset_table.drop(index=('2008-06-10-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-06-10-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-06-10-4')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-06-10-5')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-07-07-3')) #Array Error
dataset_table = dataset_table.drop(index=('2008-07-07-4')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-08-26-0')) #Array Error
dataset_table = dataset_table.drop(index=('2008-08-26-2')) #Array Error
dataset_table = dataset_table.drop(index=('2008-08-27-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-08-27-4')) #Array Error
dataset_table = dataset_table.drop(index=('2008-10-11-0')) #Array Error
dataset_table = dataset_table.drop(index=('2008-10-11-1')) #Array Error
dataset_table = dataset_table.drop(index=('2008-11-10-1')) #Array Error
dataset_table = dataset_table.drop(index=('2008-11-10-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-11-10-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2008-11-10-4')) #Array Error
dataset_table = dataset_table.drop(index=('2008-11-12-3')) #Array Error
dataset_table = dataset_table.drop(index=('2010-03-05-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2010-10-18-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2010-10-18-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2010-10-18-7')) #Array Error 
dataset_table = dataset_table.drop(index=('2010-10-28-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2010-11-22-0')) #EI calc needed
dataset_table = dataset_table.drop(index=('2010-11-22-1')) #Array Error 
dataset_table = dataset_table.drop(index=('2010-11-22-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2010-11-22-5')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-01-11-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-05-11-5')) #Array Error
dataset_table = dataset_table.drop(index=('2011-05-11-8')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-06-24-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-06-24-5')) #Array Error
dataset_table = dataset_table.drop(index=('2011-07-05-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-07-05-5')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-07-14-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-07-14-5')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-08-01-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-08-01-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-08-04-0')) #Array Error
dataset_table = dataset_table.drop(index=('2011-08-04-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-10-25-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-10-25-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-10-25-4')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-10-25-7')) #Array Error
dataset_table = dataset_table.drop(index=('2011-12-04-0')) #ei
dataset_table = dataset_table.drop(index=('2011-12-13-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2011-12-13-2')) #buffer
dataset_table = dataset_table.drop(index=('2012-01-27-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2012-01-27-5')) #Array Error 
dataset_table = dataset_table.drop(index=('2012-08-09-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2012-08-09-3')) #sta
dataset_table = dataset_table.drop(index=('2012-09-13-0')) #Array Error 
dataset_table = dataset_table.drop(index=('2012-09-18-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2012-09-18-5')) #Array Error 
dataset_table = dataset_table.drop(index=('2013-01-23-2')) #Array Error
dataset_table = dataset_table.drop(index=('2013-05-28-2')) #Array Error 
dataset_table = dataset_table.drop(index=('2013-05-28-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2013-05-28-6')) #Array Error
dataset_table = dataset_table.drop(index=('2013-06-18-4')) #Array Error
dataset_table = dataset_table.drop(index=('2013-06-18-9')) #Array Error
dataset_table = dataset_table.drop(index=('2014-04-10-0')) #Array Error
dataset_table = dataset_table.drop(index=('2014-04-15-4')) #Array Error 
dataset_table = dataset_table.drop(index=('2014-06-04-0')) #Label Error 
dataset_table = dataset_table.drop(index=('2014-06-04-3')) #Array Error 
dataset_table = dataset_table.drop(index=('2014-11-05-0')) #Buffer Error 
dataset_table = dataset_table.drop(index=('2014-11-24-1')) #Buffer Error 
dataset_table = dataset_table.drop(index=('2018-03-01-5')) #Label Error
dataset_table = dataset_table.drop(index=('2018-08-07-1')) #Label Error
dataset_table = dataset_table.drop(index=('2018-08-07-2')) #Label Error
dataset_table = dataset_table.drop(index=('2018-08-07-5')) #Label Error
dataset_table = dataset_table.drop(index=('2018-08-07-9')) #Nan Error 

In [586]:
# IF ERROR IS THROWN IN FEATURE GENERATION 
# 1) Drop in dataset_table and ct table (make sure to add to full list above
ct.dataset_table = ct.dataset_table.drop(index=('2018-08-07-9'))
ct.unit_table = ct.unit_table.drop(index=('2018-08-07-9'))
dataset_table = dataset_table.drop(index=('2018-08-07-9')) #Array Error 
# display(ct.dataset_table)
# 2) Add only one new dataset and initialize
dataset_id_new = ct.add_datasets(dataset_table.iloc[359:360,:])
ct.initialize_units_for_datasets(dataset_id_new)

In [576]:
# The CellTable lives in Cell Display Lib, a module full of functions that are useful for cell data handling

importlib.reload(cdl)
ct = cdl.CellTable()  # Initialize the core object, called the CellTable spelled 'ct'



~ Welcome to the CellTable ~experience~
~ 
~ Starting a fresh new CellTable


In [577]:
# load the dataset_table we created into the ct
dataset_ids = ct.add_datasets(dataset_table.iloc[360:, :])



~ Adding 8 datasets


In [578]:
ct.initialize_units_for_datasets(dataset_ids)

# let's look at the unit_table, where most of the analysis is stored, which we have just filled with one row for every unit.
# Each unit is also indexed by a tuple: ('piece_id','run_id','unit_id'), the last of which is the familiar Vision ID,
# the output from the spike sorter. Note that these are not consecutive, but might be arranged on the electrode array.
# the table also has columns 'unit_id' and 'dataset_id' for easy access later and to keep things organized
# the column 'valid' is for excluding cells with basic errors, like having no spikes

~ Loading new units from MultiIndex([('2018-03-01-4', '000'),
            ('2018-03-01-8', '002'),
            ('2018-04-18-3', '009'),
            ('2018-08-07-1', '002'),
            ('2018-08-07-2', '000'),
            ('2018-08-07-5', '000'),
            ('2018-08-07-6', '000'),
            ('2018-08-07-9', '000')],
           names=['piece_id', 'run_id']), currently have 8 datasets
~ Starting with unit_table having 0 units
~ Dataset ('2018-03-01-4', '000'): Loaded 803 units, sorter/alex indices 1 through 7670
~ Dataset ('2018-03-01-8', '002'): Loaded 230 units, sorter/alex indices 36 through 7758
~ Dataset ('2018-04-18-3', '009'): Loaded 372 units, sorter/alex indices 31 through 7666
~ Dataset ('2018-08-07-1', '002'): Loaded 704 units, sorter/alex indices 1 through 7668
~ Dataset ('2018-08-07-2', '000'): Loaded 763 units, sorter/alex indices 1 through 7669
~ Dataset ('2018-08-07-5', '000'): Loaded 614 units, sorter/alex indices 1 through 7667
~ Dataset ('2018-08-07-6', '000'): Loa

MultiIndex([('2018-03-01-4', '000'),
            ('2018-03-01-8', '002'),
            ('2018-04-18-3', '009'),
            ('2018-08-07-1', '002'),
            ('2018-08-07-2', '000'),
            ('2018-08-07-5', '000'),
            ('2018-08-07-6', '000'),
            ('2018-08-07-9', '000')],
           names=['piece_id', 'run_id'])

In [585]:
# Features are how we generate the parameters that each cell has, which we use for analysis.
# We'll make our own feature in a later section

# a list of all of the features we want to run per-dataset
# each of these are subclasses of Feature
features_to_generate_by_dataset = [
    feat.Feature_load_manual_labels,
    feat.Feature_load_dataset_metadata,

    # electrical
    feat_e.Feature_load_spike_times, feat_e.Feature_spikes_basic,
    feat_e.Feature_load_ei,
    feat_e.Feature_generate_acf_from_spikes,
    feat_e.Feature_spike_waveform,
    feat_e.Feature_ei_correlation_data,
    feat_e.Feature_ei_select_electrodes,
    feat_e.Feature_ei_map,

    # deep learning characteristics
    feat_dl.Feature_int_piece_id,
    feat_dl.Feature_spec_spike_waveform,
    feat_dl.Feature_spec_acf,
    feat_dl.Feature_retinal_eccentricity,
    feat_dl.Feature_map_early_ei_char,
    feat_dl.Feature_spike_char,
    feat_dl.Feature_axon_vel
    
]

# and then one feature to run overall, which sorts out the manual labels from text into integers
features_to_generate_overall = [feat.Feature_process_manual_labels]

# if the ct is very large we'll drop the big columns like STA and EI to save disk space, as they can be
# easily reloaded later as needed
drop_big_columns = ct.unit_table.shape[0] > 10000
# Removed 'ei' from big_columns_per_dataset b/c will likely use
big_columns_per_dataset = ['ei','alex_id','spike_times','spike_count','spike_waveform_maxenergy','spike_waveform_smart','ei_edge','ei_peak','ei_axon_only','ei_energy_raw','map_ei_energy','map_ei_energy_early','map_ei_energy_late']

# now generate the features we listed above, and display the resulting units_table
# check out all of those new columns, which you can use to make your analysis

ct.generate_features('all', features_to_generate_by_dataset, features_to_generate_overall,
                     force_features=False,
                     load_analysis_data=True, #I changed this line
                     ignore_errors=True,
                     drop_big_columns=False,
                     big_columns_per_dataset=big_columns_per_dataset)

display(ct.unit_table)

~ Activating per-dataset features ["<class 'features.Feature_load_manual_labels'>", "<class 'features.Feature_load_dataset_metadata'>", "<class 'features_electrical.Feature_load_spike_times'>", "<class 'features_electrical.Feature_spikes_basic'>", "<class 'features_electrical.Feature_load_ei'>", "<class 'features_electrical.Feature_generate_acf_from_spikes'>", "<class 'features_electrical.Feature_spike_waveform'>", "<class 'features_electrical.Feature_ei_correlation_data'>", "<class 'features_electrical.Feature_ei_select_electrodes'>", "<class 'features_electrical.Feature_ei_map'>", "<class 'features_DLelec.Feature_int_piece_id'>", "<class 'features_DLelec.Feature_spec_spike_waveform'>", "<class 'features_DLelec.Feature_spec_acf'>", "<class 'features_DLelec.Feature_retinal_eccentricity'>", "<class 'features_DLelec.Feature_map_early_ei_char'>", "<class 'features_DLelec.Feature_spike_char'>", "<class 'features_DLelec.Feature_axon_vel'>"]
*** timer  started
~ 

Generating features for dat

Traceback (most recent call last):
  File "/Volumes/Lab/Development/miniconda-peggyo/envs/mads3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_68017/4153831325.py", line 42, in <cell line: 42>
    ct.generate_features('all', features_to_generate_by_dataset, features_to_generate_overall,
  File "/Volumes/Lab/Users/scooler/classification/cell_display_lib.py", line 1057, in generate_features
    indices = indices[self.unit_table.loc[indices,'valid']]
  File "/Volumes/Lab/Development/miniconda-peggyo/envs/mads3/lib/python3.9/site-packages/pandas/core/indexes/multi.py", line 2084, in __getitem__
    if com.is_bool_indexer(key):
  File "/Volumes/Lab/Development/miniconda-peggyo/envs/mads3/lib/python3.9/site-packages/pandas/core/common.py", line 144, in is_bool_indexer
    raise ValueError(na_msg)
ValueError: Cannot mask with non-boolean array containing NA / NaN values

During 

In [587]:
# ct.cell_table contains just the true unique cells in each piece, with integer indices
# the CellTable has a decent deduplication algorithm implemented, which combines multiple units into single cells
# if you don't care for deduplication, just don't run it. You will still need to initialize_cell_table though,
# and you'll have one cell per unit.

ct.initialize_cell_table()
deduplication.deduplicate(ct, use_sta=False, do_combine_unit_data=False, verbose=False)

display(ct.cell_table)

Piece 2018-03-01-4 has 803 units from 1 datasets
~ failed to move label_manual
Piece 2018-03-01-8 has 230 units from 1 datasets
~ failed to move label_manual
Piece 2018-04-18-3 has 372 units from 1 datasets
~ failed to move label_manual
Piece 2018-08-07-6 has 795 units from 1 datasets
~ failed to move label_manual
~ failed to move label_manual
~ Created fresh new cell_table and reset unit_table links. Now, you may deduplicate.
Deduplicating units into cells.
~ Correlating EI between units
*** timer  started

Evaluating piece 2018-03-01-4 w/ 1 datasets, 803 units
Map dim 512, wave dim 111. Correlating EI energy first.
Correlating spike waveforms.
~ ... piece 2018-03-01-4, correlated EI maps & spike waveforms from 803 units.
*** elapsed 3s of 3s = 0.0m elapsed, of 0.2m estimated (1/4) (0.4 / sec)

Evaluating piece 2018-03-01-8 w/ 1 datasets, 230 units
Map dim 519, wave dim 71. Correlating EI energy first.
Correlating spike waveforms.
~ ... piece 2018-03-01-8, correlated EI maps & spike w

KeyError: 'label_manual_input'

In [590]:
# let's select some ON parasols from the first dataset
# indices = ct.cell_selection(types=['ON parasol'], pieces=['2018-03-01-0'])
indices = ct.cell_selection(types=['ON parasol'])

# we could also get the first dataset from the table index and use that:
# dataset_id = ct.dataset_table.index[0]
# indices = ct.cell_selection(types=['ON parasol'], pieces=[dataset_id])


# either way, we now have all of the cells we want
# Let's grab the first one and see what it has in there
id = indices[0]
print(id) # see the piece_id, cell_id
cell = ct.get_cell(id)
print(cell['piece_id'])
print(cell)

# lots of things in there!
# column names are on the left, values are on the right.
# a value starting with 'a' is an array, the following integers are the dimensions.

# Let's plot the ACF:

plt.figure()
plt.plot(cell['acf'].a)
plt.show()

# NOTE!!! you see that '.a' up there? That's important. To ease storing arrays and dicts in pandas
# dataframes, we store them inside a wrapper object, defined in file_handling.wrapper
# to access the data, just add .a to the end. That wrapper is what generates those 'a40x20x6' strings you saw.

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Volumes/Lab/Development/miniconda-peggyo/envs/mads3/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3621, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 136, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 163, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5198, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5206, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'label_manual_text'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Volumes/Lab/Development/miniconda-peggyo/envs/mads3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_68017/1402364279.py", 

In [588]:
# Drop Large Columns
# if the ct is very large we'll drop the big columns like STA and EI to save disk space, as they can be
# easily reloaded later as needed
drop_big_columns = ct.unit_table.shape[0] > 10000
# Removed 'ei' from big_columns_per_dataset b/c will likely use
big_columns_per_dataset = ['ei','alex_id','spike_times','spike_count','spike_waveform_maxenergy','spike_waveform_smart','ei_edge','ei_peak','ei_axon_only','ei_energy_raw','map_ei_energy','map_ei_energy_early','map_ei_energy_late']

# now generate the features we listed above, and display the resulting units_table
# check out all of those new columns, which you can use to make your analysis
ct.generate_features('all', features_to_generate_by_dataset, features_to_generate_overall,
                     force_features=False,
                     load_analysis_data=True,
                     ignore_errors=False,
                     drop_big_columns=True,
                     big_columns_per_dataset=big_columns_per_dataset)
display(ct.unit_table)

~ Activating per-dataset features ["<class 'features.Feature_load_manual_labels'>", "<class 'features.Feature_load_dataset_metadata'>", "<class 'features_electrical.Feature_load_spike_times'>", "<class 'features_electrical.Feature_spikes_basic'>", "<class 'features_electrical.Feature_load_ei'>", "<class 'features_electrical.Feature_generate_acf_from_spikes'>", "<class 'features_electrical.Feature_spike_waveform'>", "<class 'features_electrical.Feature_ei_correlation_data'>", "<class 'features_electrical.Feature_ei_select_electrodes'>", "<class 'features_electrical.Feature_ei_map'>", "<class 'features_DLelec.Feature_int_piece_id'>", "<class 'features_DLelec.Feature_spec_spike_waveform'>", "<class 'features_DLelec.Feature_spec_acf'>", "<class 'features_DLelec.Feature_retinal_eccentricity'>", "<class 'features_DLelec.Feature_map_early_ei_char'>", "<class 'features_DLelec.Feature_spike_char'>", "<class 'features_DLelec.Feature_axon_vel'>"]
*** timer  started
~ 

Generating features for dat

unit_id           dataset_id run_id  \
piece_id     run_id unit_id                                        
2018-03-01-4 000    1            1.0  (2018-03-01-4, 000)    000   
                    3            3.0  (2018-03-01-4, 000)    000   
                    31          31.0  (2018-03-01-4, 000)    000   
                    32          32.0  (2018-03-01-4, 000)    000   
                    46          46.0  (2018-03-01-4, 000)    000   
                    48          48.0  (2018-03-01-4, 000)    000   
                    61          61.0  (2018-03-01-4, 000)    000   
                    76          76.0  (2018-03-01-4, 000)    000   
                    106        106.0  (2018-03-01-4, 000)    000   
                    108        108.0  (2018-03-01-4, 000)    000   
                    136        136.0  (2018-03-01-4, 000)    000   
                    137        137.0  (2018-03-01-4, 000)    000   
                    166        166.0  (2018-03-01-4, 000)    000   
                    168        168.0  (2018-03-01-4, 000)    000   
                    182        182.0  (2018-03-01-4, 000)    000   
                    198        198.0  (2018-03-01-4, 000)    000   
                    199        199.0  (2018-03-01-4, 000)    000   
                    201        201.0  (2018-03-01-4, 000)    000   
                    211        211.0  (2018-03-01-4, 000)    000   
                    212        212.0  (2018-03-01-4, 000)    000   
                    226        226.0  (2018-03-01-4, 000)    000   
                    241        241.0  (2018-03-01-4, 000)    000   
                    256        256.0  (2018-03-01-4, 000)    000   
                    257        257.0  (2018-03-01-4, 000)    000   
                    259        259.0  (2018-03-01-4, 000)    000   
                    274        274.0  (2018-03-01-4, 000)    000   
                    277        277.0  (2018-03-01-4, 000)    000   
                    286        286.0  (2018-03-01-4, 000)    000   
                    287        287.0  (2018-03-01-4, 000)    000   
                    302        302.0  (2018-03-01-4, 000)    000   
                    305        305.0  (2018-03-01-4, 000)    000   
                    316        316.0  (2018-03-01-4, 000)    000   
                    317        317.0  (2018-03-01-4, 000)    000   
                    322        322.0  (2018-03-01-4, 000)    000   
                    346        346.0  (2018-03-01-4, 000)    000   
                    349        349.0  (2018-03-01-4, 000)    000   
                    362        362.0  (2018-03-01-4, 000)    000   
                    363        363.0  (2018-03-01-4, 000)    000   
                    367        367.0  (2018-03-01-4, 000)    000   
                    368        368.0  (2018-03-01-4, 000)    000   
                    391        391.0  (2018-03-01-4, 000)    000   
                    392        392.0  (2018-03-01-4, 000)    000   
                    436        436.0  (2018-03-01-4, 000)    000   
                    437        437.0  (2018-03-01-4, 000)    000   
                    438        438.0  (2018-03-01-4, 000)    000   
                    451        451.0  (2018-03-01-4, 000)    000   
                    467        467.0  (2018-03-01-4, 000)    000   
                    496        496.0  (2018-03-01-4, 000)    000   
                    498        498.0  (2018-03-01-4, 000)    000   
                    499        499.0  (2018-03-01-4, 000)    000   
                    500        500.0  (2018-03-01-4, 000)    000   
                    501        501.0  (2018-03-01-4, 000)    000   
                    502        502.0  (2018-03-01-4, 000)    000   
                    511        511.0  (2018-03-01-4, 000)    000   
                    513        513.0  (2018-03-01-4, 000)    000   
                    517        517.0  (2018-03-01-4, 000)    000   
                    527        527.0  (2018-03-01-4, 000)    000   
                 

In [589]:
# this function will save the units, cells, and dataset_table row in its own table, which everyone shares (unless you use a
# different file_root argument, which you should if you are making weird tables)
# the save_name argument is specific to you and this project, always, and saves your dataset_table in a
# single file so you can easily reload the whole set of datasets

save_name = 'train340to360'  # customize this!
scratch_file_root = '/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/' # replace my name!

save_file = 1

if save_file:
    ct.file_save_pieces(save_name=save_name, file_root=scratch_file_root)
else:
    print('NO SAVE')

~ Saving 4 pieces, to /Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/
~ Saving entire datasets table for later per-piece file loading
~ glady saved the whole dataset_table to /Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctds_train360to364.pkl
*** timer  started
~ piece_id 2018-03-01-4 with 1 runs, 803 cells, 803 units
~ Saved datasets, cells, units to pandas DataFrames at
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctd_2018-03-01-4_[d,c,u].pkl
*** elapsed 0s of 0s = 0.0m elapsed, of 0.0m estimated (1/4) (12.9 / sec)
~ piece_id 2018-03-01-8 with 1 runs, 230 cells, 230 units
~ Saved datasets, cells, units to pandas DataFrames at
/Volumes/Scratch/Users/mads/celltable_datasets/featExtractDL/train/ctd_2018-03-01-8_[d,c,u].pkl
*** elapsed 0s of 0s = 0.0m elapsed, of 0.0m estimated (2/4) (18.2 / sec)
~ piece_id 2018-04-18-3 with 1 runs, 372 cells, 372 units
~ Saved datasets, cells, units to pandas DataFrames at
/Volumes/Scratch/